线性回归的简介实现（使用框架）

In [26]:
# 生成数据集
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [27]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    # shuffle 洗牌，混淆
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [28]:
next(iter(data_iter))

[tensor([[-0.0127,  0.5131],
         [ 0.5232,  1.4856],
         [ 0.7662, -0.3994],
         [-0.8291,  0.9101],
         [-0.1053, -0.4220],
         [ 1.7371, -0.5355],
         [-0.0993, -0.2377],
         [ 1.4284, -1.5263],
         [-0.6033, -0.1275],
         [-1.5669, -0.1054]]),
 tensor([[ 2.4242],
         [ 0.1974],
         [ 7.0826],
         [-0.5495],
         [ 5.4320],
         [ 9.5052],
         [ 4.8043],
         [12.2347],
         [ 3.4279],
         [ 1.4090]])]

In [29]:
#定义模型
from torch import nn

# 只有一层线性变换 在该层线性变换中，输入特征数为2，输出的特征数为1
net = nn.Sequential(nn.Linear(2,1))

# 关于上面函数的讲解

## torch.nn

PyTorch 中的 torch.nn 包提供了很多与实现神经网络中的具体功能相关的类，这些类涵盖了深度神经网络模型在搭建和参数优化过程中的常用内容，比如神经网络中的卷积层、池化层、全连接层这类层次构造的方法、防止过拟合的参数归一化方法、Dropout 方法，还有激活函数部分的线性激活函数、非线性激活函数相关的方法，等等。

下面主要介绍三个函数：
    1. nn.Sequential函数
    2.nn.Linear函数

### torch.nn.Sequential

torch.nn.Sequential 类是 torch.nn 中的一种序列容器，通过在容器中嵌套各种实现神经网络中具体功能相关的类，来完成对神经网络模型的搭建，最主要的是，参数会按照我们定义好的序列自动传递下去。

我们可以将嵌套在容器中的各个部分看作各种不同的模块，这些模块可以自由组合。模块的加入一般有两种方式，一种是在以上代码中使用的直接嵌套，另一种是以 orderdict 有序字典的方式进行传入，这两种方式的唯一区别是：

    使用 orderdict 搭建的模型的每个模块都有我们自定义的名字；
    而前者默认使用从零开始的数字序列作为每个模块的名字；

    使用 orderdict 的示例：

    torch.nn.Sequential(OrderedDict([
        ("Line1", t.nn.Linear(input_data, hidden_layer)),
        ("ReLU", t.nn.ReLU()),
        ("Line2",t.nn.Linear(hidden_layer, output_data))
    ]))


### torch.nn.Linear 

torch.nn.Linear 类用于定义模型的线性层，即完成前面提到的不同的层之间的线性变换。
torch.nn.Linear 类接收的参数有三个，分别是输入特征数、输出特征数和是否使用偏置，设置是否使用偏置的参数是一个布尔值，默认为 True ，即使用偏置。


In [30]:
# 设置参数的初始值
#  我们通过net[0]选择网络中的第一个图层， 然后使用weight.data和bias.data方法访问参数。 我们还可以使用替换方法normal_和fill_来重写参数值。
# 我们指定每个权重参数应该从均值为0、标准差为0.01的正态分布中随机采样， 偏置参数将初始化为零
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [31]:
# 定义损失函数，这里使用MSELoss类，即L2范数
loss = nn.MSELoss()

In [32]:
# 定义优化算法，这里选择随机梯度下降法（SGD）
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

[Pytorch中常用的四种优化器SGD、Momentum、RMSProp、Adam](https://www.jianshu.com/p/1a1339c4acd7)


In [33]:
# 训练
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        # 这里进行参数优化
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000296
epoch 2, loss 0.000105
epoch 3, loss 0.000105


In [34]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-5.5981e-04, -4.5300e-06])
b的估计误差： tensor([-0.0006])
